# Unpack Data

In this notebook, we'll take the `basic` data set, use `ibmseti` Python package to convert each data file into a spectrogram.

In [1]:
from __future__ import division

# import cStringIO
# import glob
# import json
# import requests
import ibmseti
import os
import zipfile
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

In [2]:
teamname = 'Machine_Contact_Enterprise_SETI'

mydatafolder = os.path.join( os.environ['PWD'], teamname )      

# Create folder for zip to extract to
setiDataDir = mydatafolder + '/dataFinal'
if os.path.exists(setiDataDir) is False:
    os.makedirs(setiDataDir)
    
# Create folder for future spectrograms
outputpng_folder = mydatafolder + '/pngFinal'
if os.path.exists(outputpng_folder) is False:
    os.makedirs(outputpng_folder)

In [3]:
# Extract Data folder
def unzip_data(dest, zipf):
    # zipf = array of zip files     
    for i in zipf:
        zz = zipfile.ZipFile(mydatafolder + '/' + i )
        zz.extractall(dest)

        
        
unzip_data(setiDataDir, ['basic4_test_hackathon_final.zip'])

# when using the larger dataset, you will have multiple zip files. Pass an array of those zip files to unzip_data

In [4]:
# Choose your data set!
workingDataDir = setiDataDir
workingIndexFile = os.path.join( mydatafolder + '/public_list_basic_test_hackathon_final.csv' )


In [15]:
ls $mydatafolder/pngFinal

In [11]:
#Use `ibmseti`, or other methods, to draw the spectrograms

def draw_spectrogram(data):
    
    aca = ibmseti.compamp.SimCompamp(data)
    spec = aca.get_spectrogram()
    fig, ax = plt.subplots(figsize=(10, 5))   
    
    ax.imshow(np.log(spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0], cmap='gray')
    
    return fig, aca.header()

def convert_to_spectrogram_and_save(row):
    
    try:
        uuid, classification = row.split(',')
    except:
        uuid = row #this handles the test data since it doesn't have "SIGNAL_CLASSIFICATION" in index file
        classification = 'unknown: test data'

    #create path to local data file
    filename = uuid + '.dat'
    filepath = os.path.join(workingDataDir, filename)
    
    #retrieve that data file
    rawdata = open(filepath).read()
    
    
    fig, header = draw_spectrogram(rawdata)
    
    png_file_name = filename + '.png'
    fig.savefig( os.path.join(outputpng_folder, png_file_name) )
    plt.close(fig)
    
    return (filename, header, png_file_name)

In [13]:
rdd = sc.textFile(workingIndexFile, 30).filter(lambda x: x.startswith('UUID') is False) #the filter removes the header

In [14]:
rdd.count()

1498

In [17]:
results = rdd.map(convert_to_spectrogram_and_save).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 2.0 failed 10 times, most recent failure: Lost task 8.9 in stage 2.0 (TID 535, yp-spark-dal09-env5-0034): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/src/spark160master/spark-1.6.0-bin-2.6.0/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/src/spark160master/spark-1.6.0-bin-2.6.0/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/src/spark160master/spark-1.6.0-bin-2.6.0/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-11-8b7ef98d6f35>", line 29, in convert_to_spectrogram_and_save
  File "<ipython-input-11-8b7ef98

In [ ]:
results[0]